In [1]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import Token_NOAA
from config import API_Token
import calendar
#Suppress Warnings
pd.options.mode.chained_assignment = None  # default='warn'
#Display all columns
pd.set_option('display.max_columns', None)

In [2]:
# https://github.com/crvaden/NOAA_API_v2
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://cran.r-project.org/web/packages/rnoaa/rnoaa.pdf
# file:///C:/Users/15124/Downloads/GHCND_documentation.pdf
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

In [3]:
# Import the API key.
headers = {
    'accept': 'application/json',
    'authorization': API_Token
    }

# White Wine in USA API

In [4]:
#import JSON data with the API
req_white_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=white&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_white_USA = urllib.request.urlopen(req_white_USA).read().decode()
obj_white_USA = json.loads(res_white_USA)
#read json into DF
white_USA_df = pd.DataFrame.from_dict(obj_white_USA['results'], orient='columns')
# Delete unnecessary columns
white_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
##'wine_id','wine_type','classification',
##Drop NA values
# white_USA_df = white_USA_df.dropna()
#convert vintage to date type
white_USA_df['vintage']= pd.to_datetime(white_USA_df['vintage']).dt.year
white_USA_df['regions']=white_USA_df['regions'].str.get(0)
# Read the DF to a csv
white_USA_df.to_csv("White_Wine_USA.csv")
# white_USA_df.head(2)

# Weather Data By Zip Code

In [5]:
bigdf = pd.DataFrame()
date_max_temp= pd.DataFrame()
date_min_temp= pd.DataFrame()
states = { 
    'CA': {
        'zip': '97218',
        'years': ['2010', '2011']
        },
    'Wash': {
        'zip': '99336',
        'years': ['2010', '2011']
        }
    #etc...
}
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #e = json.loads(p.text)
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']

        # #get the date field from all average temperature readings
        list_max_temp = [item['date'] for item in items_MAX]
        list_min_temp = [item['date'] for item in items_MIN]
        
        
        date_max_temp['date']=[datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in list_max_temp]
        date_min_temp['date']=[datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in list_min_temp]

        df_test['date'] = date_max_temp['date']
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']

        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
        small_df = pd.concat([small_df,df_test])
    if bigdf.empty:
        bigdf = small_df
    else:
        bigdf = pd.merge(bigdf,small_df)
bigdf = bigdf.set_index('date').resample('M').mean()
# bigdf.head(2)


bigdf.reset_index(inplace=True)
bigdf['day'] = bigdf['date'].dt.day
bigdf['month'] = bigdf['date'].dt.month
bigdf['year'] = bigdf['date'].dt.year

list_months=[]
for x in bigdf['month']:
    list_months.append(calendar.month_name[x])
bigdf['months']=list_months
bigdf.drop(['date', 'day','month'], axis=1, inplace=True)
# bigdf.reindex(['year','months','avgMaxTempCA','avgMinTempCA', 'avgMaxTempWash', 'avgMinTempWash'],axis=1)

working on year 2010
working on year 2011
working on year 2010
working on year 2011


In [19]:
temp_df=pd.read_csv('Final_Temp_Weather_API.csv')
prcp_df=pd.read_csv('Final_Prcp_Weather_API.csv')

# List to iterate through

In [7]:
# calender_years=[1992,1993, 1994, 1995,1996,1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,2013, 2014,2015, 2016, 2017]
calender_years=[2010, 2011]

calender_months=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
regions=[]

In [8]:
white_USA_df.head(2)

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Ridge Vineyards, Monte Bello Chardonnay, White...",120785,Santa Cruz Mountains,White,dry,California,Usa,None,2016,False,94.39,B,3
1,"Stags' Leap Winery, Chardonnay, White, Napa Va...",129823,Napa Valley,White,dry,California,Usa,None,2016,False,89.83,A+,3


In [27]:
sorted_temp_df=temp_df[temp_df['Year']==2010]
sorted_temp_df

,avgTempNapa,avgTempColumbia,avgTempWalla,avgTempSonoma,avgTempSanta,avgTempYakima,avgTempDundee,Year,Months
216,47.944444,33.888889,43.370370,54.814815,51.740741,36.462963,46.277778,2010,January
217,50.640000,38.280000,48.600000,60.760000,53.440000,40.860000,45.960000,2010,February
218,51.928571,44.839286,56.035714,64.321429,53.660714,43.589286,47.214286,2010,March
219,53.300000,50.580000,62.360000,64.360000,54.540000,49.480000,50.120000,2010,April
220,56.590909,54.295455,65.727273,68.909091,57.522727,53.886364,53.909091,2010,May
221,65.041538,62.173077,75.807692,81.076923,62.519231,62.884615,60.269231,2010,June
222,63.790323,73.354839,87.838710,82.709677,61.516129,71.758065,67.500000,2010,July
223,64.225806,70.209677,85.225806,80.967742,62.887097,69.016129,67.403226,2010,August
224,66.883333,62.566667,75.500000,84.166667,65.150000,62.133333,63.650000,2010,September
225,60.193548,52.145161,64.032258,73.290323,60.032258,51.241935,54.532258,2010,October


In [30]:
for month in calender_months:
    white_USA_df['avgTemp'+str(month)]=0

for i in range(len(white_USA_df)):
    for month in calender_months:           
        if white_USA_df['vintage'][i]==2010:
            white_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==2010].avgTempNapa
#             white_USA_df['January'][i]=1
        else:
            white_USA_df['avgTemp'+month][i]=0
sorted_white_USA_df=white_USA_df[white_USA_df['vintage']==2010]
sorted_white_USA_df.head(2)

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count,avgTempJanuary,avgTempFebruary,avgTempMarch,avgTempApril,avgTempMay,avgTempJune,avgTempJuly,avgTempAugust,avgTempSeptember,avgTempOctober,avgTempNovember,avgTempDecember,avgPrcpJanuary,avgPrcpFebruary,avgPrcpMarch,avgPrcpApril,avgPrcpMay,avgPrcpJune,avgPrcpJuly,avgPrcpAugust,avgPrcpSeptember,avgPrcpOctober,avgPrcpNovember,avgPrcpDecember
124,"Kistler Vineyards, Hyde Vineyard Chardonnay, W...",91299,Carneros,White,dry,California,Usa,None,2010,False,94.76,C+,3,47,50,51,53,56,65,63,64,66,60,53,49,0,0,0,0,0,0,0,0,0,0,0,0
131,"Kongsgaard, The Judge Chardonnay, White, Napa ...",91591,Napa Valley,White,dry,California,Usa,None,2010,False,96.58,B,4,47,50,51,53,56,65,63,64,66,60,53,49,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
for month in calender_months:
    white_USA_df['avgPrcp'+str(month)]=0

for i in range(len(white_USA_df)):
    for month in calender_months:           
        if white_USA_df['vintage'][i]==2010:
            white_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==2010].PRCPNapa
#             white_USA_df['January'][i]=1
        else:
            white_USA_df['avgPrcp'+month][i]=0
sorted_white_USA_df=white_USA_df[white_USA_df['vintage']==2010]
sorted_white_USA_df.head(2)   

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count,avgTempJanuary,avgTempFebruary,avgTempMarch,avgTempApril,avgTempMay,avgTempJune,avgTempJuly,avgTempAugust,avgTempSeptember,avgTempOctober,avgTempNovember,avgTempDecember,avgPrcpJanuary,avgPrcpFebruary,avgPrcpMarch,avgPrcpApril,avgPrcpMay,avgPrcpJune,avgPrcpJuly,avgPrcpAugust,avgPrcpSeptember,avgPrcpOctober,avgPrcpNovember,avgPrcpDecember
124,"Kistler Vineyards, Hyde Vineyard Chardonnay, W...",91299,Carneros,White,dry,California,Usa,None,2010,False,94.76,C+,3,47,50,51,53,56,65,63,64,66,60,53,49,0,0,0,0,0,0,0,0,0,0,0,0
131,"Kongsgaard, The Judge Chardonnay, White, Napa ...",91591,Napa Valley,White,dry,California,Usa,None,2010,False,96.58,B,4,47,50,51,53,56,65,63,64,66,60,53,49,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
prcp_df.loc[prcp_df['months']=='January'].loc[prcp_df['year']==2010].PRCPNapa

216    0.389697
Name: PRCPNapa, dtype: float64

In [24]:
sorted_prcp_df=prcp_df[prcp_df['year']==2010]
sorted_prcp_df

,index,PRCPNapa,PRCPColumbia,PRCPWalla,PRCPSonoma,PRCPSanta,PRCPYakima,PRCPDundee,year,months
216,216,0.389697,0.060758,0.037273,0.360758,0.347879,0.060000,0.428788,2010,January
217,217,0.154848,0.057273,0.026818,0.160909,0.238333,0.032273,0.303182,2010,February
218,218,0.100278,0.003333,0.022361,0.093009,0.110370,0.005093,0.348750,2010,March
219,219,0.194453,0.039416,0.044781,0.105876,0.225657,0.018285,0.304927,2010,April
220,220,0.046021,0.031414,0.084188,0.076649,0.038482,0.057644,0.121885,2010,May
221,221,0.000000,0.070268,0.070804,0.000000,0.000000,0.041250,0.144375,2010,June
222,222,0.000000,0.007742,0.010323,0.000000,0.000645,0.002581,0.003226,2010,July
223,223,0.000000,0.019677,0.002903,0.000000,0.000968,0.001613,0.000000,2010,August
224,224,0.000000,0.053333,0.026333,0.000000,0.000333,0.029333,0.079333,2010,September
225,225,0.216648,0.041420,0.058807,0.245455,0.195170,0.028807,0.472159,2010,October


# Working -----------------------------------------Working

In [ ]:
for month in calender_months:
    white_USA_df['avgTemp'+str(month)]=0

for i in range(len(white_USA_df)):
    for month in calender_months:           
        if white_USA_df['vintage'][i]==2010:
            white_USA_df['avgTemp'+month][i]=bigdf.loc[bigdf['months']==month].loc[bigdf['year']==2010].avgMaxTempCA
#             white_USA_df['January'][i]=1
        else:
            white_USA_df['avgTemp'+month][i]=0
sorted_white_USA_df=white_USA_df[white_USA_df['vintage']==2010]
sorted_white_USA_df.head(2)

In [12]:
for i in range(len(white_USA_df)):
    for month, cyear in zip(calender_months,calender_years) :           
        if white_USA_df['vintage'][i]==2010:
            white_USA_df['avgTemp'+month][i]=bigdf.loc[bigdf['months']==month].loc[bigdf['year']==2010].avgMaxTempCA
        else:
            white_USA_df['avgTemp'+month][i]=0
white_USA_df

TypeError: 'str' object is not callable

# Practice

In [ ]:
data = {'Name': ['Ankit', 'Amit', 'Aishwarya', 'Priyanka'], 
                'Age': [21, 19, 20, 18], 
                'Stream': ['Math', 'Commerce', 'Arts', 'Biology'], 
                'Percentage': [88, 92, 95, 70]} 
  
# Convert the dictionary into DataFrame 
df = pd.DataFrame(data, columns = ['Name', 'Age', 'Stream', 'Percentage'])
df['A']=0
df

In [ ]:
for i in range(len(df)):
    if df['Percentage'][i]==70:
        df['Percentage'][i]=df.loc[df['Name']=='Aishwarya'].Age

for i in range(len(df)):
    if df['Percentage'][i]==20:
        df['A'][i]=1
    else:
        df['A'][i]=0

df

In [ ]:
print(bigdf.loc[bigdf['months']=='January'].loc[bigdf['year']==2010].avgMaxTempCA)
print(bigdf.loc[bigdf['months']=='February'].loc[bigdf['year']==2010].avgMaxTempCA)
print(bigdf.loc[bigdf['months']=='March'].loc[bigdf['year']==2010].avgMaxTempCA)
print(bigdf.loc[bigdf['months']=='April'].loc[bigdf['year']==2010].avgMaxTempCA)
print(bigdf.loc[bigdf['months']=='May'].loc[bigdf['year']==2010].avgMaxTempCA)
print(bigdf.loc[bigdf['months']=='June'].loc[bigdf['year']==2010].avgMaxTempCA)
print(bigdf.loc[bigdf['months']=='July'].loc[bigdf['year']==2010].avgMaxTempCA)
print(bigdf.loc[bigdf['months']=='August'].loc[bigdf['year']==2010].avgMaxTempCA)

In [ ]:
# loc is label based: 1, 2, 100--- you get rows that have a 1 or 2 or 100
# iloc is int based: 1, 2, 100---- you get the 1st, 2nd, 100th index
# bigdf.loc[bigdf['months']=='January'].loc[bigdf['year']==2010].avgMaxTempCA
x=[]
for month in calender_months:
    x.append(bigdf.loc[bigdf['months']==month].loc[bigdf['year']==2010].avgMaxTempCA)
x

In [ ]:
# #Where Method
# bigdf['avgMaxTempCA'].where(bigdf['months']=='January').where(bigdf['year']==2010)


# for i in range(len(white_USA_df)):
#     if white_USA_df['vintage'][i]==2010:
#         white_USA_df[month][i]==bigdf.loc[bigdf['year']==2010].loc[bigdf['months']==month].avgMaxTempCA
#     else:
#         white_USA_df[month][i]==0
# sorted_white_USA_df=white_USA_df[white_USA_df['vintage']==2010]
# sorted_white_USA_df.head(2)

In [ ]:
#Where Method
# bigdf['avgMaxTempCA'].where(bigdf['months']=='January').where(bigdf['year']==2010)


# for i in white_USA_df['vintage']:
#     if i == 2010:
#         white_USA_df['February']=bigdf['avgMaxTempCA'].where(bigdf['months']=='February').where(bigdf['year']==2010)
#     else:
#         white_USA_df['February']==0
# white_USA_df
# sorted_white_USA_df=white_USA_df[white_USA_df['vintage']==2010]
# sorted_white_USA_df.head(2)


for i in range(len(white_USA_df)):
    if white_USA_df.loc[i, 'vintage']==2010:
        white_USA_df.loc[i, 'January']==1
    else:
        white_USA_df.loc[i, 'January']==0
sorted_white_USA_df=white_USA_df[white_USA_df['vintage']==2010]
sorted_white_USA_df.head(2)

In [ ]:
for i in white_USA_df['vintage']:
    for month in calender_months:
        if i ==2010:
            white_USA_df[month]=bigdf['avgMaxTempCA'].where(bigdf['months']==month).where(bigdf['year']==2010)
        else:
            white_USA_df[month]=0
sorted_white_USA_df=white_USA_df[white_USA_df['vintage']==2010]
sorted_white_USA_df.head(2) 

In [ ]:
# sorted_white_USA_df=white_USA_df.sort_values(by=['vintage'], ascending=True)
sorted_white_USA_df=white_USA_df[white_USA_df['vintage']==2010]
sorted_white_USA_df.head(2)

In [ ]:
for i in range(len(white_USA_df)):
    if white_USA_df['vintage'][i]==2010:
        white_USA_df['January']=bigdf['avgMaxTempCA'].where(bigdf['months']=='January').where(bigdf['year']==2010)
    else:
        white_USA_df['January'][i]==0
sorted_white_USA_df=white_USA_df[white_USA_df['vintage']==2010]
sorted_white_USA_df.head(2)
        

# white_USA_df['January']=bigdf['avgMaxTempCA'].where(bigdf['months']=='January').where(bigdf['year']==2010)
# white_USA_df

In [ ]:
bigdf.loc[bigdf['months']==month].loc[bigdf['year']==2010].avgMaxTempCA

In [ ]:
# bigdf['avgCA']=0
# for i in range(len(bigdf)):
#     bigdf['avgCA'][i]=(bigdf['avgMaxTempCA'][i]+bigdf['avgMinTempCA'][i])/2
# bigdf


